In [5]:
import os
from typing import List, Dict, Any, TypedDict, Annotated, Literal
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

In [6]:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
import os

class RealEstateRAG:
    def __init__(self,llm,embedding_model ,openai_api_key=None):
        # Set up OpenAI API key
        #self.api_key = openai_api_key or os.getenv("OPENAI_API_KEY")
        #if not self.api_key:
        #    raise ValueError("OpenAI API key is required")

        # Initialize Neo4j connection
        self.graph = Neo4jGraph(
            url="bolt://localhost:7687",
            username="neo4j",
            password="1234neo4j"
        )

        # Initialize embeddings
        self.embeddings = embedding_model #OpenAIEmbeddings(api_key=self.api_key)

        # Initialize vector store
        # self.vector_store = Neo4jVector.from_existing_graph(
        #     embedding=self.embeddings,
        #     url="bolt://localhost:7687",
        #     username="neo4j",
        #     password="1234neo4j",
        #     search_type="hybrid",  # Combines vector + keyword search
        #     node_label="Building",  # Main node label to search
        #     text_node_properties=["description"],  # Property containing text to embed
        #     embedding_node_property="embedding"  # Property to store embeddings
        # )
        self.llm = llm
        # Initialize LLM
        # self.llm = ChatOpenAI(
        #     temperature=0,
        #     model="gpt-4",
        #     api_key=self.api_key
        # )

        # Set up the Cypher QA Chain
        self.qa_chain = self._setup_qa_chain()

    def _setup_qa_chain(self):
        # Custom prompt template for real estate queries
        CYPHER_PROMPT = """Task: Generate a Cypher query to find real estate information based on the question.

        Database Schema:
        - (Building) nodes with properties: name, description, latitude, longitude, about, in_a_nutshell, etc.
        - (Area) nodes with properties: name, description, about_, in_a_nutshell, community_overview, etc.
        - Specific amenity nodes:
        * (Transport) with properties: name, business_status, vicinity, latitude, longitude, rating, user_ratings_total
        * (Gym) with properties: name, business_status, vicinity, latitude, longitude, rating, user_ratings_total
        * (SPA) with properties: name, business_status, vicinity, latitude, longitude, rating, user_ratings_total
        * (Supermarket) with properties: name, business_status, vicinity, latitude, longitude, rating, user_ratings_total
        * (Mall) with properties: name, business_status, vicinity, latitude, longitude, rating, user_ratings_total
        * (MetroStation) with properties: name, business_status, vicinity, latitude, longitude, rating, user_ratings_total
        * (Hospital) with properties: name, business_status, vicinity, latitude, longitude, rating, user_ratings_total
        * (School) with properties: name, business_status, vicinity, latitude, longitude, rating, user_ratings_total
        * (Worship) with properties: name, business_status, vicinity, latitude, longitude, rating, user_ratings_total
        * (Restaurant) with properties: name, business_status, vicinity, latitude, longitude, rating, user_ratings_total
        * (Park) with properties: name, business_status, vicinity, latitude, longitude, rating, user_ratings_total
        * (TouristAttraction) with properties: name, business_status, vicinity, latitude, longitude, rating, user_ratings_total
        
        - Relationships: 
        * (Area)-[:CONTAINS]->(Building)
        * (Building)-[:HAS_TRANSPORT]->(Transport)
        * (Building)-[:HAS_GYM]->(Gym)
        * (Building)-[:HAS_SPA]->(SPA)
        * (Building)-[:HAS_SUPERMARKET]->(Supermarket)
        * (Building)-[:HAS_MALL]->(Mall)
        * (Building)-[:HAS_METRO]->(MetroStation)
        * (Building)-[:HAS_HOSPITAL]->(Hospital)
        * (Building)-[:HAS_SCHOOL]->(School)
        * (Building)-[:HAS_WORSHIP]->(Worship)
        * (Building)-[:HAS_RESTAURANT]->(Restaurant)
        * (Building)-[:HAS_PARK]->(Park)
        * (Building)-[:HAS_TOURIST_ATTRACTION]->(TouristAttraction)

        Question: {question}

        Important:
        - Consider spatial relationships when relevant
        - Include ratings and review counts when available
        - Limit results to reasonable numbers (5-10)
        - Filter for operational amenities using business_status='OPERATIONAL' when appropriate
        - Only generate the Cypher query, no other text

        Cypher Query:"""

        cypher_prompt = PromptTemplate(
            template=CYPHER_PROMPT,
            input_variables=["question"]
        )

        return GraphCypherQAChain.from_llm(
            llm=self.llm,
            graph=self.graph,
            cypher_prompt=cypher_prompt,
            verbose=True,
            allow_dangerous_requests=True
        )
    def answer_query(self, query: str) -> str:
        """
        Process and answer real estate queries using graph capabilities
        """
        try:
            # Use the QA chain to get the answer directly
            # No vector store needed
            response = self.qa_chain.run(query)
            return response

        except Exception as e:
            return f"Error processing query: {str(e)}"


In [7]:
import os
from openai import AzureOpenAI
from azure.core.credentials import AzureKeyCredential  # Add this import
from dotenv import load_dotenv
load_dotenv()
model_name = "text-embedding-3-small"
deployment = "text-embedding-3-small"

api_version = "2024-02-01"
endpoint = os.getenv("AZURE_OPENAI_URL")
subscription_key = os.getenv("AZURE_OPENAI_KEY")
embedding_client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=endpoint,
    api_key=subscription_key
)

In [8]:
from langchain_openai import AzureChatOpenAI
from langchain_core.tools import tool

model_name = "gpt-4o"
deployment = "azure-gpt-4o"

api_version = "2024-12-01-preview"
endpoint = os.getenv("AZURE_OPENAI_URL")
subscription_key = os.getenv("AZURE_OPENAI_KEY")
llm = AzureChatOpenAI(
    azure_deployment=deployment,
    azure_endpoint=endpoint,
    api_version=api_version,  
    temperature=0.7,
    api_key=subscription_key
)

In [9]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

# Connect to your SQLite database
db = SQLDatabase.from_uri("sqlite:///real_estate.db")

# Define your database schema and field descriptions as a system prompt
schema_description = """
    Database: real_estate.db (SQLite)

    Table: listings_buy (Listings about proprties ready to buy)
    Columns: 
    - title (TEXT): Property title/description
    - price (TEXT): Listed price
    - type (TEXT): Property type (Apartment, Villa, etc.)
    - beds (REAL): Number of bedrooms
    - studio (TEXT): Whether it's a studio apartment
    - baths (INTEGER): Number of bathrooms
    - area (TEXT): Property area in square feet
    - location (TEXT): Full location string
    - payment (TEXT): Payment plan
    - agent_plan (TEXT): Agent information
    - building_name (TEXT): Name of the building
    - street_name (TEXT): Street name
    - area_name (TEXT): Area/neighborhood name
    - verified_date (TIMESTAMP): Date when listing was verified
    - handover quater (TEXT): Quarter of handover
    - handover year (TEXT): Year of handover
    - offplan type (TEXT): Off-plan property type
    - type of sale (TEXT): Initial Sale or Resale

    Table: listings_rent (Listings about proprties ready to rent)
    Columns:
    - title (TEXT): Property title/description
    - price (TEXT): Rental price
    - type (TEXT): Property type (Apartment, Villa, etc.)
    - beds (REAL): Number of bedrooms
    - studio (TEXT): Whether it's a studio apartment
    - baths (INTEGER): Number of bathrooms
    - area (TEXT): Property area in square feet
    - location (TEXT): Full location string
    - agent_plan (TEXT): Agent information
    - building_name (TEXT): Name of the building
    - street_name (TEXT): Street name
    - area_name (TEXT): Area/neighborhood name
    - verified_date (TIMESTAMP): Date when listing was verified

    Table: transactions_buy (Transactions  about properties that were sold)
    Columns:
    - DATE (TIMESTAMP): Transaction date
    - LOCATION (TEXT): Full location string
    - Type (TEXT): Property type
    - Beds (TEXT): Number of bedrooms
    - BUILT-UP AREA (TEXT): Built-up area in square feet
    - FLOOR (TEXT): Floor number
    - BUILT-UP (TEXT): Additional built-up area information
    - PLOT (TEXT): Plot information
    - building_name (TEXT): Name of the building
    - area_name (TEXT): Area/neighborhood name
    - PRICE (TEXT): Transaction price
    - Info (TEXT): Additional information

    Table: transactions_rent Transactions  about properties that were rented)
    Columns:
    - START DATE (TIMESTAMP): Rental start date
    - LOCATION (TEXT): Full location string
    - Price (TEXT): Rental price
    - Type (TEXT): Property type
    - BEDS (TEXT): Number of bedrooms
    - AREA (SQFT) (TEXT): Area in square feet
    - FLOOR (TEXT): Floor number
    - building_name (TEXT): Name of the building
    - area_name (TEXT): Area/neighborhood name
    - DURATION(Months) (TEXT): Rental duration in months
    - Info (TEXT): Additional information (NEW or RENEWAL)
"""

# Initialize the language model with the schema information
#llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# Create a custom prompt template that includes the schema
system_template = f"""You are an AI assistant that translates natural language to SQL.
{schema_description}

When translating natural language to SQL:
1. Only use tables and columns that exist in the schema
2. Be precise with column names and table relationships
3. Use appropriate SQL syntax for SQLite
4. Return only the SQL query without additional explanation unless asked
"""

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_message_prompt = HumanMessagePromptTemplate.from_template("{input}")

chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt
])

# Create the SQL toolkit with the custom prompt
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# Create the SQL agent with the custom system prompt
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    system_message=system_template
)

In [10]:
import os
from typing import List, Dict, Any, TypedDict, Annotated, Literal
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

# Configure your API keys
#os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

# Define the state that will be passed between nodes
class AgentState(TypedDict):
    query: str
    plan: List[str]
    current_step: int
    graph_results: Dict[str, Any]
    sql_results: Dict[str, Any]
    final_answer: str
    messages: List[Any]
    _last_agent:str

In [3]:
def planning_agent(state: AgentState) -> AgentState:
    """Agent that breaks down complex property queries into sub-tasks and coordinates the workflow"""
    query = state["query"]
    print("inside planning agent.")

    if not state["plan"]:
        planning_prompt = f"""
        You are a planning agent for a Dubai real estate assistant. Break down the user's query into specific sub-tasks.
        You have access to:
        1. GraphRAG Agent: Information about Dubai areas, buildings, amenities like parks hospital, and location features
        2. SQL Agent: Current property listings (buy/rent) and historical transaction data
        
        User query: "{query}"
        
        Create a step-by-step plan to answer this query. Each step MUST start with either:
        - "Call GraphRAG Agent:" followed by the specific task
        - "Call SQL Agent:" followed by the specific task
        
        Format each step on a new line with a dash (-) prefix.
        """
        
        response = llm.invoke(planning_prompt)
        
        # Parse the response to extract the plan steps
        plan_steps = [step.strip() for step in response.content.split("\n") if step.strip().startswith("-")]
        if not plan_steps:
            plan_steps = [response.content]
        print(plan_steps)
        return {
            **state,  # Include all state properties
            "_last_agent": "planning_agent",
            "plan": plan_steps,
            "current_step": 0,
            "messages": state["messages"] + [AIMessage(content=f"I'll help you find good value properties in Dubai. My plan is:\n" + "\n".join(plan_steps))]
        }
    
    # If we're returning to the planner after an agent has run
    else:
        # Check if we have all the information we need
        has_graph_data = bool(state["graph_results"])
        has_sql_data = bool(state["sql_results"])
        all_steps_completed = state["current_step"] >= len(state["plan"])
        
        # Decide what to do next based on available information
        if all_steps_completed and has_graph_data and has_sql_data:
            # We have everything we need, ready for synthesis
            return {
                **state,  # Include all state properties
                "_last_agent": "planning_agent",
                "messages": state["messages"] + [AIMessage(content="I've gathered all the necessary information. Now I'll provide you with recommendations.")]
            }
        elif state["current_step"] < len(state["plan"]):
            # We still have steps to complete
            current_step = state["plan"][state["current_step"]]
            
            # Determine which agent should handle the current step based on explicit prefixes
            if "Call GraphRAG Agent:" in current_step and not has_graph_data:
                # Need area information
                # Prepare context for the GraphRAG agent if SQL data is available
                context_for_graph = ""
                if state["sql_results"]:
                    context_for_graph = f"SQL data context: {state['sql_results'].get('content', '')}"
                
                return {
                    **state,  # Include all state properties
                    "_last_agent": "planning_agent",
                    "context_for_next_agent": context_for_graph,
                    "messages": state["messages"] + [AIMessage(content=f"Now I'll get information about the area: {current_step}")]
                }
            elif "Call SQL Agent:" in current_step and not has_sql_data:
                # Need property data
                # Prepare context for the SQL agent if graph data is available
                context_for_sql = ""
                if state["graph_results"]:
                    context_for_sql = f"Graph data context: {state['graph_results'].get('content', '')}"
                
                return {
                    **state,  # Include all state properties
                    "_last_agent": "planning_agent",
                    "context_for_next_agent": context_for_sql,
                    "messages": state["messages"] + [AIMessage(content=f"Now I'll get property data: {current_step}")]
                }
            else:
                # Move to the next step
                return {
                    **state,
                    "current_step": state["current_step"] + 1,
                    "messages": state["messages"] + [AIMessage(content=f"Moving to the next step: {state['plan'][state['current_step']]}")]
                }
        else:
            # Fallback - get any missing information
            if not has_graph_data:
                return {
                    **state,
                    "messages": state["messages"] + [AIMessage(content="I need to get information about the area first.")]
                }
            elif not has_sql_data:
                return {
                    **state,
                    "messages": state["messages"] + [AIMessage(content="I need to get property data next.")]
                }
            else:
                # Ready for synthesis
                return {
                    **state,
                    "messages": state["messages"] + [AIMessage(content="I've gathered all the necessary information. Now I'll provide you with recommendations.")]
                }

def graph_rag_agent(state: AgentState) -> AgentState:
    """Agent that provides information about Dubai areas, buildings and amenities"""
    query = state["query"]
    current_step = state["plan"][state["current_step"]] if state["current_step"] < len(state["plan"]) else ""
    
    # Get context from previous agents if available
    context = state.get("context_for_next_agent", "")
    
    # Extract the specific task from the current step
    task = current_step.replace("Call GraphRAG Agent:", "").strip() if "Call GraphRAG Agent:" in current_step else query
    
    # Create an enhanced query with context
    enhanced_query = f"{task}\n\n{context}" if context else task
    
    # Use the RealEstateRAG class with the enhanced query
    rag = RealEstateRAG(llm, embedding_client)
    response = rag.answer_query(enhanced_query)
    
    return {
        **state,  # Preserve all state properties
        "_last_agent": "graph_rag_agent",
        "graph_results": {"content": response, "task": task},
        "current_step": state["current_step"] + 1,
        "messages": state["messages"] + [AIMessage(content=f"{response}")]
    }

def sql_agent(state: AgentState) -> AgentState:
    """Agent that provides property listings and transaction data"""
    query = state["query"]
    current_step = state["plan"][state["current_step"]] if state["current_step"] < len(state["plan"]) else ""
    
    # Extract the specific task from the current step
    task = current_step.replace("Call SQL Agent:", "").strip() if "Call SQL Agent:" in current_step else query
    
    # Get context from previous agents if available
    context = state.get("context_for_next_agent", "")
    
    # Get area information from GraphRAG agent if available
    area_info = state["graph_results"].get("content", "No area information available yet.")
    
    # Create an enhanced SQL prompt with context from other agents
    sql_prompt = f"""
    You are a Dubai real estate data agent with access to:
    - Current property listings (buy and rent)
    - Historical transaction data for rent and buy transactions
    - Price per square foot statistics
    - Rental yield information
    
    User query: "{query}"
    Current task: "{task}"
    
    {context}
    
    Area information: {area_info}
    
    When searching via area names use LIKE operator to widen the search results
    """
    
    response = agent.invoke(sql_prompt)
    
    return {
        **state,  # Preserve all state properties
        "_last_agent": "sql_agent",
        "sql_results": {"content": response.content if hasattr(response, 'content') else str(response), "task": task},
        "messages": state["messages"] + [AIMessage(content=f"{response.content if hasattr(response, 'content') else str(response)}")]
    }

def synthesis_agent(state: AgentState) -> AgentState:
    """Agent that synthesizes information from other agents to provide final recommendations"""
    query = state["query"]
    graph_results = state["graph_results"].get("content", "")
    graph_task = state["graph_results"].get("task", "")
    sql_results = state["sql_results"].get("content", "")
    sql_task = state["sql_results"].get("task", "")
    
    synthesis_prompt = f"""
    You are a Dubai real estate advisor helping clients find good value properties.
    
    User query: "{query}"
    
    Area information task: {graph_task}
    Area information results: {graph_results}
    
    Property data task: {sql_task}
    Property data results: {sql_results}
    
    Based on this information, provide a comprehensive answer that highlights good value properties
    to buy or rent in Dubai. Include specific recommendations and explain why they represent good value.
    Make sure to address all aspects of the user's original query.
    """
    
    response = llm.invoke(synthesis_prompt)
    
    return {
        **state,
        "final_answer": response.content,
        "messages": state["messages"] + [AIMessage(content=response.content)],
        "_last_agent": "synthesis_agent"
    }

# Define the hub router function
def hub_router(state: AgentState) -> Literal["graph_rag_agent", "sql_agent", "synthesis_agent", "planning_agent", "END"]:
    """Routes between the planner (hub) and specialized agents (spokes)"""
    # If we're just starting, go to the planner
    print("INSIDE HUB_ROUTER")
    print(len(state["plan"]))
    if len(state["plan"]) == 0: #not state["plan"]:
        return "planning_agent"
    print("last_agent",state.get("_last_agent"))
    # If we just came from the planner
    if state.get("_last_agent") == "planning_agent":
        current_step = state["plan"][state["current_step"]] if state["current_step"] < len(state["plan"]) else ""
        
        # Check if we're ready for synthesis
        if state["current_step"] >= len(state["plan"]) and state["graph_results"] and state["sql_results"]:
            return "synthesis_agent"
            
        # Determine which agent to call based on the explicit agent prefix in the current step
        if "Call GraphRAG Agent:" in current_step and not state["graph_results"]:
            return "graph_rag_agent"
        elif "Call SQL Agent:" in current_step and not state["sql_results"]:
            return "sql_agent"
        elif not state["graph_results"]:
            return "graph_rag_agent"
        elif not state["sql_results"]:
            return "sql_agent"
        else:
            return "synthesis_agent"
    
    # If we just came from a specialized agent, go back to the planner
    elif state.get("_last_agent") in ["graph_rag_agent", "sql_agent"]:
        return "planning_agent"
    
    print("Inside default fallabck")
    return "planning_agent"

# Build the hub-and-spoke graph
def build_hub_spoke_graph():
    workflow = StateGraph(AgentState)
    
    # Add nodes
    workflow.add_node("planning_agent", planning_agent)
    workflow.add_node("graph_rag_agent", graph_rag_agent)
    workflow.add_node("sql_agent", sql_agent)
    workflow.add_node("synthesis_agent", synthesis_agent)
    
    # Add conditional edges using the router
    workflow.add_conditional_edges(
        "planning_agent",
        hub_router,
        {
            "graph_rag_agent": "graph_rag_agent",
            "sql_agent": "sql_agent",
            "synthesis_agent": "synthesis_agent",
            "planning_agent": "planning_agent",
            "END": END
        }
    )
    
    workflow.add_conditional_edges(
        "graph_rag_agent",
        hub_router,
        {
            "planning_agent": "planning_agent"
        }
    )
    
    workflow.add_conditional_edges(
        "sql_agent",
        hub_router,
        {
            "planning_agent": "planning_agent",
        }
    )
    
    workflow.add_edge("synthesis_agent",END)
    
    # Set the entry point
    workflow.set_entry_point("planning_agent")
    
    return workflow.compile()
# Create the agent executor with hub-and-spoke architecture
dubai_property_agent = build_hub_spoke_graph()


In [11]:
# Example usage
def get_property_recommendations(query: str):
    initial_state = {
        "query": query,
        "plan": [],
        "current_step": 0,
        "graph_results": {},
        "sql_results": {},
        "final_answer": "",
        "messages": [HumanMessage(content=query)]
    }
    
    result = dubai_property_agent.invoke(initial_state)
    return result["final_answer"]

# Test the agent
if __name__ == "__main__":
    query = "I'm looking for a affordable 2-bedroom apartment with good rated schools"
    recommendation = get_property_recommendations(query)
    print(recommendation)

inside planning agent.
['- Call GraphRAG Agent: Identify areas in Dubai with good-rated schools.', '- Call SQL Agent: Fetch current property listings for 2-bedroom apartments in the identified areas.', '- Call SQL Agent: Filter the property listings to show only affordable options based on historical transaction data and current market trends.', '- Call GraphRAG Agent: Verify the proximity of the affordable 2-bedroom apartments to the identified good-rated schools.', "- Call SQL Agent: Provide the final list of affordable 2-bedroom apartments near good-rated schools for the user's consideration."]
INSIDE HUB_ROUTER
5
last_agent planning_agent


/tmp/ipykernel_205509/3405858691.py:16: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  self.graph = Neo4jGraph(
/tmp/ipykernel_205509/3405858691.py:113: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = self.qa_chain.run(query)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (area:Area)-[:CONTAINS]->(building:Building)-[:HAS_SCHOOL]->(school:School)
WHERE school.business_status = 'OPERATIONAL' AND school.rating >= 4
RETURN area.name AS area, school.name AS school_name, school.rating AS rating, school.user_ratings_total AS review_count
ORDER BY school.rating DESC, school.user_ratings_total DESC
LIMIT 10

Full Context:
[{'area': 'Dubai Sports City', 'school_name': 'Hubaishy’s', 'rating': 5.0, 'review_count': 9.0}, {'area': 'Dubai Sports City', 'school_name': 'Hubaishy’s', 'rating': 5.0, 'review_count': 9.0}, {'area': 'Dubai Sports City', 'school_name': 'Hubaishy’s', 'rating': 5.0, 'review_count': 9.0}, {'area': 'Dubai Sports City', 'school_name': 'Hubaishy’s', 'rating': 5.0, 'review_count': 9.0}, {'area': 'Dubai Sports City', 'school_name': 'Hubaishy’s', 'rating': 5.0, 'review_count': 9.0}, {'area': 'Dubai Sports City', 'school_name': 'Hubaishy’s', 'rating': 5.0, 'review_count': 9.0}

In [27]:
import streamlit as st
import os
from typing import List, Dict, Any
from langchain_core.messages import HumanMessage, AIMessage
import time

# Import your existing agent components
#from main import get_property_recommendations

# Set page configuration
st.set_page_config(
    page_title="Dubai Property Advisor",
    page_icon="🏢",
    layout="wide"
)

# Initialize session state for chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Header and description
st.title("🏢 Dubai Property Advisor")
st.markdown("""
This AI assistant helps you find good value properties in Dubai based on your requirements.
It combines knowledge about Dubai areas, amenities, and current property listings.
""")

# Sidebar with options
with st.sidebar:
    st.header("About")
    st.markdown("""
    This tool uses:
    - Neo4j graph database for area and amenity information
    - SQLite database for property listings and transactions
    - LangGraph for agent orchestration
    """)
    
    st.header("Example Queries")
    st.markdown("""
    - I'm looking for a 2-bedroom apartment in Downtown Dubai with good ROI
    - What are good value properties near highly rated schools?
    - Find me apartments in JLT with good metro access under 1.5M AED
    - What areas have the best rental yields for 1-bedroom apartments?
    """)

# Main chat interface
st.header("Chat with the Dubai Property Advisor")

# Display chat history
for message in st.session_state.messages:
    if message["role"] == "user":
        with st.chat_message("user"):
            st.markdown(message["content"])
    else:
        with st.chat_message("assistant"):
            st.markdown(message["content"])

# Chat input
user_query = st.chat_input("Ask about Dubai properties...")

# Process user input
if user_query:
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": user_query})
    
    # Display user message
    with st.chat_message("user"):
        st.markdown(user_query)
    
    # Display assistant response with a thinking indicator
    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        message_placeholder.markdown("🤔 Thinking...")
        
        # Process the query through your agent
        try:
            with st.spinner("Processing your request..."):
                # Get the recommendation
                recommendation = get_property_recommendations(user_query)
                
                # Update the message placeholder with the full response
                message_placeholder.markdown(recommendation)
                
                # Add assistant response to chat history
                st.session_state.messages.append({"role": "assistant", "content": recommendation})
        except Exception as e:
            error_message = f"Sorry, I encountered an error: {str(e)}"
            message_placeholder.markdown(error_message)
            st.session_state.messages.append({"role": "assistant", "content": error_message})

# Add a footer
st.markdown("---")
st.markdown("*Dubai Property Advisor - Powered by LangGraph and LangChain*")

2025-04-23 16:53:26.222 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-23 16:53:26.224 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-23 16:53:26.225 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-04-23 16:53:26.225 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-23 16:53:26.226 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-23 16:53:26.227 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

DeltaGenerator()